In [1]:
import xml.etree.ElementTree as ET
import csv
import ast
import json
import flatten_json
from flatten_json import flatten
from pandas.io.json import json_normalize
import xmltodict
import pandas as pd
import os
import numpy as np

# Useful Function to Parse XML

In [2]:
# step 1 parse
def process_xml_dict(xml_file):
    with open(xml_file) as fd:
        doc = xmltodict.parse(fd.read())
    return doc

In [3]:
# step 2: choose sections to work on 
def process_question_tag(doc):
    if 'SurveyDefinition' in doc:
        df = doc['SurveyDefinition']['Questions']['Question']
        return df
    else:
        return ''

In [4]:
def process_response_tag(doc, field):
    if field in doc:
        df = doc[field]
        return df
    else:
        return ''
    

In [5]:
# step 3 flatten file
def process_flattening_save_results(df):
    df_new = to_dict(df)
    dic_flattened = (flatten(d) for d in df_new)
    df_flat = pd.DataFrame(dic_flattened)
    return df_flat

In [6]:
def to_dict(config):
    
    #Nested OrderedDict to normal dict.
  
    return json.loads(json.dumps(config))

In [7]:
os.getcwd()

'/Users/ljiang/Downloads/New repos/BAT_XML'

# import and Parse

In [12]:

xml_file="65f75eb4-fbaa-40d1-9536-00adc265fe36.xml"
#xml_file="XMLtest.xml"
doc=process_xml_dict(xml_file)
ordered_dict=process_question_tag(doc)
lst2=to_dict(ordered_dict)
lst2[0:5]

[{'@QuestionID': 'QID293',
  'Type': 'DB',
  'Selector': 'TB',
  'SubSelector': None,
  'QuestionText': '<p><span style="color:#e74c3c;"><b><u>YOU CAN TRANSLATE THIS QUESTIONNAIRE BY CLICKING ENGLISH[US] in the top right hand corner&nbsp;</u></b></span> <img src="https://exiger.co1.qualtrics.com/CP/Graphic.php?IM=IM_9HsGZTYrHf8O7qd" style="width: 30px; height: 31px;" />&nbsp; <img src="https://exiger.co1.qualtrics.com/CP/Graphic.php?IM=IM_9HsGZTYrHf8O7qd" style="width: 30px; height: 31px;" /> <img src="https://exiger.co1.qualtrics.com/CP/Graphic.php?IM=IM_9HsGZTYrHf8O7qd" style="width: 30px; height: 31px;" /></p>\n\n<p>&nbsp;</p>\n\n<p><b><u>Overview of the questionnaire</u></b></p>\n\n<p>The questionnaire consists of five sections.</p>\n\n<p>The first section on&nbsp;&quot;Organisation Information&quot;&nbsp;must&nbsp;be completed first before proceeding to answer any other sections.&nbsp;This is because specific questions in following sections will be generated based on your answers 

# Find QID with Choices Not "None"

In [13]:
## QA and correct: find choices not "None"
index_c=[]
index_n=[]
choices=[]
nones=[]

for i in range(len(lst2)):
    if (bool(lst2[i]["Choices"]) ):
        choices.append(lst2[i]["Choices"])
        index_c.append(lst2[i]["@QuestionID"])
       
    else:    
        nones.append("none")
        index_n.append((lst2[i]["@QuestionID"]))
        
            
list_of_tuples = list(zip(index_c,choices))
df = pd.DataFrame(list_of_tuples, columns = ['index', 'Choices']) 
df[0:20]      



,index,Choices
0,QID6,"{'Choice': {'@ID': '1', '@Recode': '1', 'Descr..."
1,QID7,"{'Choice': {'@ID': '1', '@Recode': '1', 'Descr..."
2,QID9,"{'Choice': [{'@ID': '1', '@Recode': '1', 'Desc..."
3,QID10,"{'Choice': [{'@ID': '1', '@Recode': '1', 'Desc..."
4,QID12,"{'Choice': [{'@ID': '1', '@Recode': '1', 'Desc..."
5,QID19,"{'Choice': [{'@ID': '1', '@Recode': '1', 'Desc..."
6,QID21,"{'Choice': [{'@ID': '1', '@Recode': '1', 'Desc..."
7,QID23,"{'Choice': [{'@ID': '251', '@Recode': '251', '..."
8,QID26,"{'Choice': [{'@ID': '1', '@Recode': '1', 'Desc..."
9,QID28,"{'Choice': [{'@ID': '1', '@Recode': '1', 'Desc..."


In [14]:
## some QA code
## lst2[96]
#'QID29'in index_n
#len(index_n)+len(index_c)

In [15]:
# df1=df.transpose()
# df1
#df["Choices"]


# Convert Choices into Dataframe

In [16]:
#### did QA,create a flat file for choices
#index1=[]
lst7,lst8=[],[]
lst5,lst6=[],[]
lst11,lst22=[],[]
for i in range(len(df["Choices"])):
    if type(df["Choices"][i]["Choice"])==dict:
        for key, item in (df["Choices"][i]["Choice"]).items():
            if (key=="@ID"or key=="Description"):
                lst5.append(key)
                lst6.append(item)
                lst22.append(df["index"][i])
                
    else:
        for m in range(len(df["Choices"][i]["Choice"])):
            for key, item in (df["Choices"][i]["Choice"])[m].items():
                if (key=="@ID"or key=="Description"):
                    lst7.append(key)
                    lst8.append(item)
                    lst11.append(df["index"][i])

        
list_of_tuples1 = list(zip(lst22,lst5,lst6))
df_1 = pd.DataFrame(list_of_tuples1, columns = ['index', 'Category','value']) 
df_1   

list_of_tuples2 = list(zip(lst11,lst7,lst8))
df_2 = pd.DataFrame(list_of_tuples2, columns = ['index', 'Category','value']) 
df_2[0:20]

,index,Category,value
0,QID9,@ID,1
1,QID9,Description,Yes
2,QID9,@ID,2
3,QID9,Description,No
4,QID10,@ID,1
5,QID10,Description,Yes
6,QID10,@ID,2
7,QID10,Description,No
8,QID12,@ID,1
9,QID12,Description,Corporate entity - Please tick this box if you...


In [17]:
final=pd.concat([df_1, df_2], ignore_index=True)
final1=final.transpose()
final1

,0,1,2,3,4,5,6,7,8,9,...,18300,18301,18302,18303,18304,18305,18306,18307,18308,18309
index,QID6,QID6,QID7,QID7,QID9,QID9,QID9,QID9,QID10,QID10,...,QID73,QID73,QID75,QID75,QID75,QID75,QID77,QID77,QID77,QID77
Category,@ID,Description,@ID,Description,@ID,Description,@ID,Description,@ID,Description,...,@ID,Description,@ID,Description,@ID,Description,@ID,Description,@ID,Description
value,1,Yes,1,Yes,1,Yes,2,No,1,Yes,...,2,No,1,Yes,2,No,1,Yes,2,No


# Pivot

In [22]:
## import choices flat file
c=pd.read_excel('choices fix.xlsx')
## import original unmapped file
data=pd.read_excel('results-1-8 concat.xlsx', encoding='utf-8')
data.columns = data.iloc[0]
b=c.groupby(['QID','Category']).agg({'value':'unique'}).unstack().reset_index()
b.columns = ['_'.join(tup).rstrip('_') for tup in b.columns.values]
b=b.reset_index()
b[0:20]


,index,QID,value_@ID,value_Description
0,0,Q10,"[1, 2]","[Yes, No]"
1,1,Q102,"[1, 2]","[Yes, No]"
2,2,Q104,"[1, 2]","[Yes, No]"
3,3,Q106,"[1, 2]","[Yes, No]"
4,4,Q108,"[1, 2]","[Yes, No]"
5,5,Q110,"[1, 2]","[Yes, No]"
6,6,Q112,"[1, 2]","[Yes, No]"
7,7,Q114,"[1, 2]","[Yes, No]"
8,8,Q116,"[1, 2]","[Yes, No]"
9,9,Q118,"[1, 2]","[Yes, No]"


# Slice the dataset/Groupby/Mapping


In [25]:
## slice the dataset, only responses do the mapping
data1=data[2:4425]
title=data[0:2]

## pair the values 
## transform into the dict format
#### map my_dict to replace each time


for i in range(0,len(b)):
    ## use mapping file QID as the original data file index
    Q=b['QID'][i]
    my_dict=dict(zip(b['value_@ID'][i], b['value_Description'][i]))
    
    #### map my_dict to replace
    data1[Q] = data1[Q].map(my_dict)
    
## concat 
final = pd.concat([title, data1], ignore_index=True, sort =False)
final[0:10]


/Users/ljiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,attributes,Q293,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,...,Q93,Q94,Q95,Q96,Q73,Q74,Q75,Q76,Q77,Q78
0,attributes,Q293,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,...,Q93,Q94,Q95,Q96,Q73,Q74,Q75,Q76,Q77,Q78
1,questions,YOU CAN TRANSLATE THIS QUESTIONNAIRE BY CLICKI...,This section covers information about your com...,1. Official authorisation,1.1 Your full name (The name of the individual...,1.2 Your role/position in the organisation,1.3 Your email address,1.4 I confirm that I have the authority and am...,1.5 I confirm that the information provided in...,2. Third party claims,...,Please provide an explanation as to why this a...,Do the terms and conditions in place include a...,Please provide details of the commission fee a...,Please provide an explanation as to why this a...,20. Has your organisation or any individuals a...,Please provide details of the charges [We requ...,21. Do any managerial staff or shareholders ha...,Please provide details of the connection [If y...,21. Do you have any non-contractual connection...,Please provide details of the connection [If y...
2,1,1,1,1,Nicholas Ouma Odhiambo,Senior Manager RMS Client Service,Nicholas.Odhiambo@nielsen.com,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,1,1,1,KADUYU IMMACULATE,PAYABLE AND TREASURY ACCOUNTANT,ikaduyu@ug.nationmedia.com,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,1,1,1,JASON MUTUGI,KEY ACCOUNT MANAGER,"jason@vajas.com, sales@vajas.com",NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4,1,1,1,JOSEPH KABUGI KARANJA,CHIEF EXECUTIVE,joe.kabugi@capacityoutsourcing.co.ke,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5,1,1,1,DAVE LUNGAHO SIGANGA,MANAGING PARTNER,lungaho@siganga.co.ke/dlsiganga@gmail.com,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6,1,1,1,James onyango omondi,technical manager/ director,planetboiler@yahoo.com,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,7,1,1,1,Caleb Owino,Managing Director,c.owino@fireworks-advertising.com,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,8,1,1,1,Crispin Mbogo,Managing Director,Crispin.Mbogo@droplex.com,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
#final.to_excel("final.xlsx")
